# Install Dependencies

In [26]:
!pip install "stable-baselines3[extra]"

DEPRECATION: Loading egg at /opt/homebrew/Cellar/gpgme/1.22.0/lib/python3.11/site-packages/gpg-1.22.0-py3.11-macosx-14-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [27]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Load Environment

In [28]:
environment_name = 'CartPole-v1'
env = gym.make(environment_name)#, render_mode="human")

In [29]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()  # state = observations??
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()  # action space= Discrete(2)
        n_state, reward, done, info, _ = env.step(action)  # step returns next set of observation, reward, and bool: episode done?
        score += reward
    print(f"Episode: {episode} Score: {score}")

env.close()

Episode: 1 Score: 21.0
Episode: 2 Score: 12.0
Episode: 3 Score: 11.0
Episode: 4 Score: 16.0
Episode: 5 Score: 14.0


/opt/homebrew/lib/python3.11/site-packages/gym/envs/classic_control/cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
/opt/homebrew/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


# Understand the Environment

In [30]:
env.action_space.sample()
# Read the docs on what they mean, but
# 0: Push cart to the left
# 1: Push cart to the right

0

In [31]:
env.action_space

Discrete(2)

In [32]:
env.observation_space.sample()
# Read the docs on what they mean, but
# Cart Position, Cart Velocity, Pole Angle, Pole Velocity At Tip

array([ 4.0130649e+00,  5.0866600e+37,  3.9905172e-02, -1.7480549e+38],
      dtype=float32)

In [33]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

# 3. Train an RL Model

In [34]:
log_path = os.path.join('Training', 'Logs')

In [35]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)  # model is the agent

Using cpu device


/opt/homebrew/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [36]:
#PPO??

In [37]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_9
-----------------------------
| time/              |      |
|    fps             | 4208 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2974        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008793583 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00172    |
|    learning_rate        | 0.0003      |
|    loss                 | 8.66        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.018      |
|    value_loss           | 54.5        |
-----------------------------------------
---

# 4. Save and Reload Model

In [38]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_CartPole')
print(PPO_Path)

Training/Saved Models/PPO_Model_CartPole


In [39]:
model.save(PPO_Path)

In [40]:
del model

In [41]:
model = PPO.load(PPO_Path, env=env)

# 5. Evaluate

In [42]:
env = gym.make(environment_name)#, render_mode="human")
env = DummyVecEnv([lambda: env])
evaluate_policy(model, env, n_eval_episodes=10, render=True)
# (avg reward over nb of episodes, st deviation of the reward)

/opt/homebrew/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:234: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(493.3, 20.1)

In [43]:
env.close()

# 6. Test Model

In [44]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()  # state = observations??
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _states = model.predict(obs)  # now using model
        obs, reward, done, info = env.step(action)  # step returns next set of observation, reward, and bool: episode done?
        score += reward
    print(f"Episode: {episode} Score: {score}")

Episode: 1 Score: [500.]
Episode: 2 Score: [500.]
Episode: 3 Score: [500.]
Episode: 4 Score: [293.]
Episode: 5 Score: [500.]


# 7. Viewing Logs in Tensorboard

In [45]:
training_log_path = os.path.join(log_path, 'PPO_5')

In [25]:
!tensorboard --logdir={training_log_path}

2023-10-21 12:36:01.154500: F tensorflow/c/experimental/stream_executor/stream_executor.cc:751] Non-OK-status: stream_executor::MultiPlatformManager::RegisterPlatform( std::move(cplatform)) status: INTERNAL: platform is already registered with name: "METAL"


8. Adding a Callback to the Training Process

In [48]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [49]:
save_path = os.path.join('Training', 'Saved Models')

In [50]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, callback_on_new_best=stop_callback, eval_freq=10000, best_model_save_path=save_path, verbose=1)

In [51]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [52]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_10
-----------------------------
| time/              |      |
|    fps             | 6004 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 3896        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009136958 |
|    clip_fraction        | 0.117       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000682   |
|    learning_rate        | 0.0003      |
|    loss                 | 5.71        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0187     |
|    value_loss           | 45.9        |
-----------------------------------------
--

/opt/homebrew/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


9. Changing policies

In [53]:
net_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]  # vf = value function, 128 units in each layer // pi = custom actor-critic policy

In [54]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

Using cpu device


/opt/homebrew/lib/python3.11/site-packages/stable_baselines3/common/policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [55]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_11
-----------------------------
| time/              |      |
|    fps             | 4541 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2297        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013136639 |
|    clip_fraction        | 0.178       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | -0.0016     |
|    learning_rate        | 0.0003      |
|    loss                 | 3.15        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0197     |
|    value_loss           | 18.3        |
-----------------------------------------
--

10. Using an Alternate Algorithm

In [57]:
from stable_baselines3 import DQN

In [58]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [59]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.967    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 11479    |
|    time_elapsed     | 0        |
|    total_timesteps  | 70       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.935    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 14589    |
|    time_elapsed     | 0        |
|    total_timesteps  | 136      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.904    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 16257    |
|    time_elapsed     | 0        |
|    total_timesteps  | 202      |
----------------------------------
------------------------

In [ ]:
model.save()